In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Trainer (has labels)
# trainer_df = pd.read_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
learner_tweets_df = pd.read_csv('datasets/cleaned_hatespeech_offensive.tsv', sep="\t")



In [2]:
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
# learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.8 else 0) # threshold the average values

# sample_size = 80000
# positive_ratio = 0.75

# Select the most confident positive values
# semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.8].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
# semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.2].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

# semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])
# semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)

semi_tweets = learner_tweets_df['tweet'].values
semi_labels = learner_tweets_df['label'].values


In [3]:
learner_tweets_df.head(10)

,id,label,tweet
0,0,0,rt mayasolovely as a woman you shouldnt compla...
1,1,1,rt mleew boy dats coldtyga dwn bad for cuffin ...
2,2,1,rt urkindofbrand dawg rt sbabylife you ever fu...
3,3,1,rt c_g_anderson viva_based she look like a tranny
4,4,1,rt shenikaroberts the shit you hear about me m...
5,5,1,t_madison_x the shit just blows meclaim you so...
6,6,1,__brighterdays i can not just sit up and hate ...
7,7,1,selfiequeenbri cause im tired of you big bitch...
8,8,1,and you might not get ya bitch back and thats ...
9,9,1,rhythmixx_ hobbies include fighting mariam bitch


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
learner_dataset = TweetDataset(semi_tweets, semi_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
learner_loader = DataLoader(learner_dataset, batch_size=6, shuffle=False)

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch import optim
from tqdm import tqdm

num_epochs = 1

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


model = BertForSequenceClassification.from_pretrained('models/SOLID_80ksemi_OLIDBERT_3', num_labels=2)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
model.train()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(learner_loader)
    num_batches = len(learner_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|          | 0/4131 [00:00<?, ?it/s]/tmp/ipykernel_5325/1995336368.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 4131/4131 [03:29<00:00, 19.73it/s]


In [7]:
model.save_pretrained('models/SOLID_finetuneHSO_1')